# Mestre do Mario Kart
Você e seus amigos estão disputando um campeonato de Mario Kart chamado o “Mestre do Mario Kart”. Neste campeonato,
um jogador começa como “mestre do Mario Kart” e os outros como desafiantes, que enfrentam o mestre a cada rodada. 

- Em
uma rodada, um desafiante joga contra o então “mestre do Mario Kart” e precisa vencer 2 partidas seguidas para se tornar o
novo “mestre do Mario Kart”; 
- caso não consiga, ele retorna ao final da fila de desafiantes. 
- Se o mestre é derrotado, o
desafiante se torna o novo “mestre do Mario Kart” e o perdedor entra no final da fila de desafiantes. 
- Um competidor é
declarado vencedor do campeonato se ele, enquanto “mestre do Mario Kart”, venceu todos os outros competidores
consecutivamente.

## Nível de habilidade

Cada competidor possui um nível de habilidade, que é um número inteiro de 1 a 99. 

Suponha que, em uma partida disputada
entre competidores de níveis $x$ e $y$, a probabilidade de vitória do competidor de nível $x$ é dada por $x/(x+y)$ e a do competidor de
nível $y$ é $y/(x+y)$.

## Solicitação

- Para uma lista de campeonatos possíveis, disponibilizada no arquivo campeonatos.txt, enviado junto ao enunciado do
desafio, pedimos que você nos envie um arquivo no formato .txt com a **probabilidade de que o primeiro “mestre do
Mario Kart” seja o vencedor para cada um dos campeonatos, com precisão de pelo menos 6 casas decimais**, em que
cada linha do arquivo contém a probabilidade do campeonato correspondente. 

- Caso você não consiga determinar a
probabilidade de algum campeonato, pedimos que preencha a linha do campeonato com 0, de forma a facilitar o processo de
correção.

In [2]:
using LinearAlgebra, SparseArrays, Combinatorics

In [3]:

# Probability that A(ability x) loses against B(ability y)
# therefore, there will be a probability p^2 
# that B will become the new Mario Kart Master
prob_lose(x, y) = y // (x+y)

prob_lose (generic function with 1 method)

In [4]:
struct MarkovState{T<:Integer}
    # Stores the player sequence
    # in which the first element is the Mario Kart Master,
    # the second element is the challenger(first one in the queue)
    # and so on...
    player_sequence::Vector{T}
    # The respective abilities of each player
    ability_sequence::Vector{T}
    # The current amount of times the Mario Kart Master has won
    w::T

    function MarkovState{T}(player_sequence::Vector{T}, H::Vector{T}, w::T) where T <: Integer
        new(player_sequence, H[player_sequence], w)
    end
end

MarkovState(player_sequence, H, w::T) where T <: Integer = MarkovState{T}(player_sequence, H, w)

# generates all possible markov states for a given n
function markov_states(n, H)
    # initial player sequence, e.g. abcd or 1234
    canonical_sequence = 1:n
    # all permutations of player sequences
    p_seqs = permutations(canonical_sequence)
    # amount of times the Mario Kart Master has won
    w_seq = 0:(n-2)

    vec([MarkovState(p_seq, H, w) for p_seq in p_seqs, w in w_seq])
end

markov_states (generic function with 1 method)

In [5]:
# abilities of the players(indexed in order)
#H = [1,1,1,1]
H = fill(1,6) # H is for 'habilidade'
n = length(H) # number of players
MS = markov_states(n, H) # there are n! × (n-1) markov states
m = length(MS)

282240

In [6]:
# S matrix m × m
rows_S = Int64[]
cols_S = Int64[]
vals_S = Float64[]

# T matrix m × n
rows_T = Int64[]
cols_T = Int64[]
vals_T = Float64[]

blksize = factorial(n)
  
for i in 1:m
    s1 = MS[i]
    p1 = s1.player_sequence
    x = s1.ability_sequence[1] # master's ability
    y = s1.ability_sequence[2] # challenger's ability

    # Probability that the challenger will become the new master
    # Note: when this state transition occurs, the new state
    # has w = 0, i.e. the amount of times the master has won is reset to 0
    b = vcat(p1[2], p1[3:end], p1[1])
    j = nthperm(b)
    push!(rows_S, i)
    push!(cols_S, j)
    push!(vals_S, prob_lose(x, y)^2)

    
    #@assert (nthperm(p1) + blksize * s1.w) == i
    if s1.w < n - 2
        # Probability that the master will keep his title
        # Note: when this state transition occurs, the new state
        # has w += 1, i.e. the amount of times the master has won is increased by 1
        a = vcat(p1[1], p1[3:end], p1[2])
        j = nthperm(a) + (s1.w + 1) * blksize
        push!(rows_S, i)
        push!(cols_S, j)
        push!(vals_S, 1 - prob_lose(x, y)^2)
    elseif s1.w == n - 2
        # Probability that the master will transition to the final state(absorption)
        jT = p1[1]
        push!(rows_T, i)
        push!(cols_T, jT)
        push!(vals_T, 1 - prob_lose(x, y)^2)
    end
end

# S is a m × m matrix of 
# state transition probabilities
S = sparse(rows_S, cols_S, vals_S, m, m)

282240×282240 SparseMatrixCSC{Float64, Int64} with 524160 stored entries:
⣲⣩⣽⡶⢶⣚⢻⣄⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⣭⣷⠦⢿⣛⣯⠀⠉⢿⣄⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠷⣾⣟⣻⣍⡭⠀⠀⠀⠉⢷⣤⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠶⢼⣛⣻⣭⣷⠀⠀⠀⠀⠀⠘⢷⣤⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⢿⣛⣧⣭⣷⠾⠀⠀⠀⠀⠀⠀⠀⠙⠷⣦⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠛⣯⣽⣶⣮⡛⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢷⣆⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⣻⣬⡽⡶⢿⣛⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢻⣆⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⣭⣷⠶⣟⣛⣯⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠉⢿⣄⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⣓⢾⠿⣿⣥⣍⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠉⢷⣤⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠶⢾⡛⣻⣭⣷⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠘⢷⣤⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⢿⣛⣯⣭⡷⠶⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠘⠷⣦⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⣉⢛⣿⣶⡷⢯⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢷⣆⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⣻⣭⣝⠶⢿⣛⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢻⣆⠀⠀⠀⠀⠀⠀⠀⠀
⣭⣷⠾⣟⡛⣯⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢿⣄⡀⠀⠀⠀⠀⠀
⣤⡻⠿⣟⣻⣥⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠉⢷⣤⡀⠀⠀⠀
⡶⢿⣋⣻⣭⣷⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠘⢷⣤⠀⠀
⢿⣛⣯⣭⡝⠶⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠘⠷⣦
⣒⣙⣯⣽⡿⢷⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠉
⣻⣭⣧⠶⢿⣛⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠭⠷⠾⣟⣋⡻⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀

In [7]:
# T: matrix which has the transition probabilities
# to the final states
T = sparse(rows_T, cols_T, vals_T, m, n);

In [8]:
IS = Float64.(1.0I - S)

282240×282240 SparseMatrixCSC{Float64, Int64} with 806400 stored entries:
⣳⣭⣽⡶⢶⣚⢻⣄⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⣭⣷⠷⢿⣛⣯⠀⠉⢿⣄⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠷⣾⣟⣻⣝⣭⠀⠀⠀⠉⢷⣤⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠶⢼⣛⣻⣭⣷⠑⢄⠀⠀⠀⠘⢷⣤⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⢿⣛⣧⣭⣷⠾⠀⠀⠑⢄⠀⠀⠀⠙⠷⣦⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠛⣯⣽⣶⣮⡛⠀⠀⠀⠀⠑⢄⠀⠀⠀⠙⢷⣆⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⣻⣬⡽⡶⢿⣛⠀⠀⠀⠀⠀⠀⠑⢄⠀⠀⠀⠙⢻⣆⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⣭⣷⠶⣟⣛⣯⠀⠀⠀⠀⠀⠀⠀⠀⠑⢄⠀⠀⠀⠉⢿⣄⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⣓⢾⠿⣿⣥⣍⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠑⢄⠀⠀⠀⠉⢷⣤⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠶⢾⡛⣻⣭⣷⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠑⢄⠀⠀⠀⠘⢷⣤⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⢿⣛⣯⣭⡷⠶⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠑⢄⠀⠀⠀⠘⠷⣦⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⣉⢛⣿⣶⡷⢯⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠑⢄⠀⠀⠀⠙⢷⣆⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⣻⣭⣝⠶⢿⣛⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠑⢄⠀⠀⠀⠙⢻⣆⠀⠀⠀⠀⠀⠀⠀⠀
⣭⣷⠾⣟⡛⣯⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠑⢄⠀⠀⠀⠙⢿⣄⡀⠀⠀⠀⠀⠀
⣤⡻⠿⣟⣻⣥⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠑⢄⠀⠀⠀⠉⢷⣤⡀⠀⠀⠀
⡶⢿⣋⣻⣭⣷⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠑⢄⠀⠀⠀⠘⢷⣤⠀⠀
⢿⣛⣯⣭⡝⠶⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠑⢄⠀⠀⠀⠘⠷⣦
⣒⣙⣯⣽⡿⢷⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠑⢄⠀⠀⠀⠉
⣻⣭⣧⠶⢿⣛⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠑⢄⠀⠀
⠭⠷⠾⣟⣋⡻⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠑⢄

In [9]:
soln = IS \ collect(T)

In [ ]:
soln[1,1]

0.5065812082824597

In [7]:
S = [0 0 1//4 3//4 0 0
    1//4 0 0 0 0 3//4
    0 1//4 0 0 3//4 0
    0 0 1//4 0 0 0
    1//4 0 0 0 0 0
    0 1//4 0 0 0 0]

T = [0 0
    0 0
    0 0
    3//4 0
    0 3//4
    0 3//4]

6×2 Matrix{Rational{Int64}}:
 0//1  0//1
 0//1  0//1
 0//1  0//1
 3//4  0//1
 0//1  3//4
 0//1  3//4

In [8]:
R = (I - S)
X = inv(R) * T

6×2 Matrix{Rational{Int64}}:
 208//327  119//327
  64//327  263//327
  55//327  272//327
 259//327   68//327
  52//327  275//327
  16//327  311//327